In [ ]:
# Load libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split # Import train_test_split function
import numpy as np
import gensim
from scipy.sparse import hstack
import utils


w2v_model = gensim.models.word2vec.Word2Vec.load("data/updated_google_corpus")

In [ ]:
TESTING_MODE = False
data = utils.load_data("./data/train.csv",is_train= TESTING_MODE)
print("Loaded {0} examples with {1} columns".format(data.shape[0],data.shape[1]))

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
one_hot_encoding = utils.one_hot_encoding(data,["country","region_1","region_2","province","taster_name","variety"])

In [ ]:
description_vecs = utils.text_to_embeddings(data["description"], w2v_model=w2v_model)

In [ ]:
X = hstack([description_vecs,one_hot_encoding,data[["points"]]]).tocsr()
y = data["price"]
if not TESTING_MODE:
    X_test = X[175000:]
    y_test = y[175000:]
    X = X[:175000]
    y = y[:175000]

In [ ]:
# convert to <if True> if you want to search for the parameters
if False:
    params = {
        
    }

    lr = LinearRegression()
    grid_search = utils.train_grid_search(lr, X, y, params)
    print(grid_search.best_estimator_)

In [ ]:
%%time
lr = LinearRegression()
lr, scores = utils.train_kfold(lr, X, y)

In [ ]:
if not TESTING_MODE:
    ids = data["id"][175000:]
    y_pred = lr.predict(X_test)
    # removing outliers
    y_pred[y_pred>2500] = data["price"].mean()
    y_pred[y_pred<0]=data["price"].mean()
    result = pd.DataFrame()
    result['id'] = ids
    result["price"] = y_pred
    result.to_csv("data/out_lr.csv",index=False)